# Cayley-Dickson Construction Applied to Qi Definition

*Version 1*

My original implementation of Gaussian integers included two classes, ``Zi`` and ``Qi``, where, for example, ``Zi(2, -7)`` represents a Gaussian integer, and ``Qi(-2/3, 4/5)`` represents a Gaussian rational.

I'd like to extend this code to include rational-valued quaternions and octonions using the Cayley-Dickson construction.

The [Cayley-Dickson construction](https://en.wikipedia.org/wiki/Cayley%E2%80%93Dickson_construction) is a process by which one can use a recursive definition of conjugation together with a recursive definition of multiplication to use...
* pairs of real numbers ($\mathbb{R}$) to create complex numbers,
* pairs of complex numbers ($\mathbb{C}$) to create quaternions,
* pairs of quaternions ($\mathbb{H}$) to create octonions,
* pairs of octonions ($\mathbb{O}$) to create sedenions ($\mathbb{S}$), and so on.

For more specifics, see my write-up about the Cayley-Dickson construction [at this link](https://abstract-algebra.readthedocs.io/en/latest/55_cayley_dickson.html).

In [1]:
from cayley_dickson_alg import Zi, SetScalarMult
from random import randint, seed
from fractions import Fraction
from functools import wraps

In [25]:
def gaussian_rational(fnc):
    """For use as a property that casts an argument into Gaussian rational."""
    @wraps(fnc)
    def gaussian_rational_wrapper(arg, num):
        qi = to_gaussian_rational(num)
        return fnc(arg, qi)
    return gaussian_rational_wrapper

class Qi:
    """Gaussian Rational Number Class"""

    __MAX_DENOMINATOR = 1_000_000

    def __init__(self, re=None, im=None):

        # CLAUSE AA --------------------------------------------------------
        # re is a str, float, int, or Fraction; and
        # im is a str, float, int, Fraction, or None

        if isinstance(re, (str, float, int, Fraction)):
            if isinstance(re, Fraction):
                self.__re = re
            else:
                self.__re = Fraction(re)
            if im is None:
                self.__im = Fraction(0)
            elif isinstance(im, (str, float, int, Fraction)):
                if isinstance(im, Fraction):
                    self.__im = im
                else:
                    self.__im = Fraction(im)
            else:
                raise Exception(f"Inputs incompatible: {re} and {im}")

        # CLAUSE BA --------------------------------------------------------
        # re is a complex; and
        # im is complex, a Qi, or None

        elif isinstance(re, complex):
            if im is None:
                self.__re = Fraction(re.real)
                self.__im = Fraction(re.imag)
            elif isinstance(im, (Qi, complex)):
                self.__re = Qi(re.real, re.imag)
                self.__im = Qi(im.real, im.imag)
            else:
                raise Exception(f"Inputs incompatible: {re} and {im}")

        # CLAUSE CA --------------------------------------------------------
        # re is a Qi, and im is complex, a Qi, or None

        elif isinstance(re, Qi):
            if im is None:
                self.__re = re.real
                self.__im = re.imag
            elif isinstance(im, (complex, Qi)):
                self.__re = Qi(re)
                self.__im = Qi(im)
            else:
                raise Exception(f"Inputs incompatible: {re} and {im}")

        # CLAUSE DA --------------------------------------------------------
        # re is a Zi, and im is a Zi, or None

        elif isinstance(re, Zi):
            if im is None:
                if re.is_complex():
                    self.__re = Fraction(re.real)
                    self.__im = Fraction(re.imag)
                else:
                    self.__re = Qi(re.real)
                    self.__im = Qi(re.imag)
            elif isinstance(im, Zi) and im.order() == re.order():
                    self.__re = Qi(re)
                    self.__im = Qi(im)
            else:
                raise Exception(f"Inputs incompatible: {re} and {im}")

        # CLAUSE EA --------------------------------------------------------
        # re is a list or tuple of numbers with length equal to a
        # power of 2, and im is None, or it is a tuple or list
        # similar to the one input for re.
        elif isinstance(re, (tuple, list)):
            z = Zi.from_array(re)
            if im is None:
                self.__re = z.real
                self.__im = z.imag
            elif isinstance(im, (tuple, list)) and len(im) == len(re):
                w = Zi.from_array(im)
                self.__re = z
                self.__im = w
            else:
                raise Exception(f"Inputs incompatible: {re} and {im}")

        # CLAUSE FA --------------------------------------------------------
        # Both re and im are None

        elif re is None:
            self.__re = 0
            if im is None:
                self.__im = 0
            else:
                raise Exception(f"If re is None, then im must be None. But im = {im}")
        else:
            raise Exception(f"Unexpected combination of input types: {re} and {im}")

    @classmethod
    def max_denominator(cls, denom=None):
        if denom is None:
            return cls.__MAX_DENOMINATOR
        elif isinstance(denom, int) and denom > 1:
            cls.__MAX_DENOMINATOR = denom
            return cls.__MAX_DENOMINATOR
        else:
            raise ValueError(f"Maximum denominator, {denom}, must be an integer > 1")

    @property
    def real(self) -> Fraction:
        return self.__re

    @property
    def imag(self) -> Fraction:
        return self.__im

    def __repr__(self):
        return f"{self.__class__.__name__}({self.__re}, {self.__im})"

#    def __str__(self):
#        if self.__im < 0:
#            return f"({self.__re}{self.__im}j)"
#        else:
#            return f"({self.__re}+{self.__im}j)"

    def __neg__(self):
        """Negate this Qi."""
        return Qi(-self.__re, -self.__im)

    def __eq__(self, other):
        """Return True if this Qi equals other."""
        return (self.__re == other.real) and (self.__im == other.imag)

    def __ne__(self, other):
        """Return True if this Qi does NOT equal other."""
        return (self.__re != other.real) or (self.__im != other.imag)

    @gaussian_rational
    def __add__(self, other):
        return Qi(self.__re + other.real, self.__im + other.imag)
    
    def __sub__(self, other):
        return Zi(self.__re - other.real, self.__im - other.imag)

## Generate Unittests

The following code is used to write a draft version of unit tests for the Qi class **init** method.

The output is just a fist-cut at a complete coverage set, because not all the printed unit tests will be correct, or even run, so careful checking and editing of the output is required.

In [3]:
import itertools

def print_Qi_unittests(inputs1, inputs2):
    """Print a draft version of unit tests for the Qi class init method.
    The output must be carefully checked and corrected, if necessary.
    Also, there may be exceptions where input cases are not yet supported.
    Look for the text '<<< ERROR >>>'.

    input1 and input2 should be lists of (key, value) tuples, where each
    key is a string to be used in a comment line indicating the value's type.
    """
    for x, y in itertools.product(inputs1, inputs2):
        typ1 = x[0]; typ2 = y[0]
        val1 = x[1]; val2 = y[1]
        print(f"\n        # {typ1} - {typ2}")
        try:
            if val2 is not None:
                print(f"        self.assertEqual(Qi({repr(val1)}, {repr(val2)}), {repr(Qi(val1, val2))})")
            else:
                print(f"        self.assertEqual(Qi({repr(val1)}), {repr(Qi(val1))})")
        except Exception as exc:
            print(f"        # <<< ERROR >>> {exc}")

The output of the cell below will be a complete test case.

To run it, first copy it into a separate cell and then run the cell.

In [35]:
header = """
from unittest import TestCase, TextTestRunner, defaultTestLoader

class TestQi(TestCase):

    def setUp(self) -> None:
        seed(42)

    def test_constructor(self):
        #-------------------
        # re type - im type
        #-------------------"""

in1 = [('str', '1/2'), ('float', 0.5), ('int', -3), ('Fraction', Fraction(1, 2))]
in2 = [('str', '3/4'), ('float', 0.75), ('int', 7), ('Fraction', Fraction(3, 8)), ('None', None)]
in3 = [('complex', (-1.5+2j)), ('Qi', Qi('1/2', '3/4')), ('Zi', Zi(3, -7))]
in4 = [('complex', (3-0.75j)), ('Qi', Qi('1/4', '3/4')), ('Zi', Zi(-2, 5))]

print(header)
print_Qi_unittests(in1, in2)
print_Qi_unittests(in3, in4)
print("\n# END OF FILE")


from unittest import TestCase, TextTestRunner, defaultTestLoader

class TestQi(TestCase):

    def setUp(self) -> None:
        seed(42)

    def test_constructor(self):
        #-------------------
        # re type - im type
        #-------------------

        # str - str
        self.assertEqual(Qi('1/2', '3/4'), Qi(1/2, 3/4))

        # str - float
        self.assertEqual(Qi('1/2', 0.75), Qi(1/2, 3/4))

        # str - int
        self.assertEqual(Qi('1/2', 7), Qi(1/2, 7))

        # str - Fraction
        self.assertEqual(Qi('1/2', Fraction(3, 8)), Qi(1/2, 3/8))

        # str - None
        self.assertEqual(Qi('1/2'), Qi(1/2, 0))

        # float - str
        self.assertEqual(Qi(0.5, '3/4'), Qi(1/2, 3/4))

        # float - float
        self.assertEqual(Qi(0.5, 0.75), Qi(1/2, 3/4))

        # float - int
        self.assertEqual(Qi(0.5, 7), Qi(1/2, 7))

        # float - Fraction
        self.assertEqual(Qi(0.5, Fraction(3, 8)), Qi(1/2, 3/8))

        # float - None
       

In [34]:
from unittest import TestCase, TextTestRunner, defaultTestLoader

class TestQi(TestCase):

    def setUp(self) -> None:
        seed(42)

    def test_constructor(self):
        #-------------------
        # re type - im type
        #-------------------

        # str - str
        self.assertEqual(Qi('1/2', '3/4'), Qi(1/2, 3/4))

        # str - float
        self.assertEqual(Qi('1/2', 0.75), Qi(1/2, 3/4))

        # str - int
        self.assertEqual(Qi('1/2', 7), Qi(1/2, 7))

        # str - Fraction
        self.assertEqual(Qi('1/2', Fraction(3, 8)), Qi(1/2, 3/8))

        # str - None
        self.assertEqual(Qi('1/2'), Qi(1/2, 0))

        # float - str
        self.assertEqual(Qi(0.5, '3/4'), Qi(1/2, 3/4))

        # float - float
        self.assertEqual(Qi(0.5, 0.75), Qi(1/2, 3/4))

        # float - int
        self.assertEqual(Qi(0.5, 7), Qi(1/2, 7))

        # float - Fraction
        self.assertEqual(Qi(0.5, Fraction(3, 8)), Qi(1/2, 3/8))

        # float - None
        self.assertEqual(Qi(0.5), Qi(1/2, 0))

        # int - str
        self.assertEqual(Qi(-3, '3/4'), Qi(-3, 3/4))

        # int - float
        self.assertEqual(Qi(-3, 0.75), Qi(-3, 3/4))

        # int - int
        self.assertEqual(Qi(-3, 7), Qi(-3, 7))

        # int - Fraction
        self.assertEqual(Qi(-3, Fraction(3, 8)), Qi(-3, 3/8))

        # int - None
        self.assertEqual(Qi(-3), Qi(-3, 0))

        # Fraction - str
        self.assertEqual(Qi(Fraction(1, 2), '3/4'), Qi(1/2, 3/4))

        # Fraction - float
        self.assertEqual(Qi(Fraction(1, 2), 0.75), Qi(1/2, 3/4))

        # Fraction - int
        self.assertEqual(Qi(Fraction(1, 2), 7), Qi(1/2, 7))

        # Fraction - Fraction
        self.assertEqual(Qi(Fraction(1, 2), Fraction(3, 8)), Qi(1/2, 3/8))

        # Fraction - None
        self.assertEqual(Qi(Fraction(1, 2)), Qi(1/2, 0))

        # complex - complex
        self.assertEqual(Qi((-1.5+2j), (3-0.75j)), Qi(Qi(-3/2, 2), Qi(3, -3/4)))

        # complex - Qi
        self.assertEqual(Qi((-1.5+2j), Qi(1/4, 3/4)), Qi(Qi(-3/2, 2), Qi(1/4, 3/4)))

        # complex - Zi
        # <<< ERROR >>> Inputs incompatible: (-1.5+2j) and (-2+5j)

        # Qi - complex
        self.assertEqual(Qi(Qi(1/2, 3/4), (3-0.75j)), Qi(Qi(1/2, 3/4), Qi(3, -3/4)))

        # Qi - Qi
        self.assertEqual(Qi(Qi(1/2, 3/4), Qi(1/4, 3/4)), Qi(Qi(1/2, 3/4), Qi(1/4, 3/4)))

        # Qi - Zi
        # <<< ERROR >>> Inputs incompatible: Qi(1/2, 3/4) and (-2+5j)

        # Zi - complex
        # <<< ERROR >>> Inputs incompatible: (3-7j) and (3-0.75j)

        # Zi - Qi
        # <<< ERROR >>> Inputs incompatible: (3-7j) and Qi(1/4, 3/4)

        # Zi - Zi
        self.assertEqual(Qi(Zi(3, -7), Zi(-2, 5)), Qi(3, -7))

# END OF FILE

In [18]:
TextTestRunner(verbosity=2).run(defaultTestLoader.loadTestsFromTestCase(TestQi))

test_constructor (__main__.TestQi.test_constructor) ... FAIL

FAIL: test_constructor (__main__.TestQi.test_constructor)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/var/folders/n6/_xg9_wkd7mg3w6sxqfh7z55h0000gn/T/ipykernel_2047/549633708.py", line 98, in test_constructor
    self.assertEqual(Qi(Zi(3, -7), Zi(-2, 5)), Qi(3, -7))
    ~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AssertionError: Qi(Qi(Qi(3, 0), Qi(-7, 0)), Qi(Qi(-2, 0), Qi(5, 0))) != Qi(3, -7)

----------------------------------------------------------------------
Ran 1 test in 0.004s

FAILED (failures=1)


<unittest.runner.TextTestResult run=1 errors=0 failures=1>

In [14]:
x0 = Zi(1, 2)
x0

Zi(1, 2)

In [15]:
x0.is_complex()

True

In [8]:
Qi(Zi(1, 2), Zi(3, 4))

Qi(1, 2)

In [29]:
foo = Qi(Zi(3, -7), Zi(1, 2))
foo

Qi(Qi(3, -7), Qi(1, 2))

In [33]:
foo.real.real

Fraction(3, 1)

In [32]:
foo.imag.imag

Fraction(2, 1)